# Example 2: Markov Stability applied to directed graph

In [1]:
import networkx as nx
from pygenstability import run, plotting, constructors
from pygenstability.optimal_scales import identify_optimal_scales
import numpy as np
from scipy import sparse
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
import cmath
import math
from pylab import *
from matplotlib import rc, rcParams
import networkx as nx
import csv   
import os

In [7]:
def Reverse(lst):
    new_lst = lst[::-1]
    return new_lst

def Connector(Q):
    D = nx.to_networkx_graph(Q,create_using=nx.DiGraph())
    Isolate_list=list(nx.isolates(D))
    if len(Isolate_list)>0:
        for i in Isolate_list:
            if i==0:
                Q[i+1,i]=0.0001
            else:
                Q[i-1,i]=0.0001
    del D
    return Q

def kNN(A,N,k):
    np.fill_diagonal(A,0)
    # print(max(A[:,4]))
    # A=np.where(A > 0.09, 1, 0)

    # W.sort(reverse=True)
    B1 = np.zeros((N, N))
    for i in range(N):
        W=sorted(A[i,:],reverse=True)
    #     print( W[k])
        B1[i,:]=np.where(A[i,:] > W[k], 1, 0)

    # B=np.multiply(B1,A)
    # print(W[k])
    # print(A[20,1:20])
    # print(B[20,1:20])


    C1 = np.zeros((N, N))
    for i in range(N):
        W=sorted(A[:,i],reverse=True)
    #     print( W[k])
        C1[:,i]=np.where(A[:,i] > W[k], 1, 0)
    # C=np.multiply(C1,A)
    Q1=B1+C1    
    Q2=np.where(Q1 > .9 , 1, 0) 

    Q=np.multiply(Q2,A)
    # del A
    del B1
    del C1
    del Q1
    del Q2 
    Connector(Q)

    return Q



def PRA_partition(A,Q,D,all_results):
    N=len(Q)
    Detail_Balance_Q=[]
    for i in range(N):
        for j in range(N):
            if (i>j and (Q[i,j]+Q[j,i])>0):
                Detail_Balance_Q.append(abs(Q[i,j]-Q[j,i])/(Q[i,j]+Q[j,i]))   

    #######################################################
    selected_partitions=all_results['selected_partitions']

    Di_Between=[]
    Di_Within=[]
#     print(Repository.Reverse(selected_partitions))
    for t_opt in Reverse(selected_partitions):

        B=D.copy()
        Between=D.copy()
        Within=D.copy()
 
        Community=all_results['community_id'][t_opt]
        Community=Community.tolist()
 
        ######## set node attrbute #################################################################################
        for node in B.nodes:
            B.nodes[node]["community"] = Community[node]
            
        ######## Creating a subgraph using the node atterbute ######################################################

        for x in range(max(Community)+1):
            selected_nodes = [n for n,v in B.nodes(data=True) if v["community"] == x]  
            # print (selected_nodes)
            globals()['G%s' % x]= B.subgraph(selected_nodes)

        ######## Creating the network between and within the communities ##########################################

        for x in range(max(Community)+1):
            List_edge=list(globals()['G%s' % x].edges())
            Between.remove_edges_from(List_edge)

        List_edge1=list(Between.edges())
        Within.remove_edges_from(List_edge1)

        ######## Ploting the Histogram ##############################################################################
        Wi=nx.to_numpy_array(Within)
        Be=nx.to_numpy_array(Between)
 
        Detail_Balance_Be=[]
        Detail_Balance_Wi=[]

        for i in range(N):
            for j in range(N):
                if (i>j and (Be[i,j]+  Be[j,i])>0):
        #             print(i,j,Be[i,j])
                    Detail_Balance_Be.append(abs(Be[i,j]-Be[j,i])/(Be[i,j]+Be[j,i]))
                if (i>j and (Wi[i,j]+Wi[j,i])>0):
        #             print(i,j,Wi[i,j])`
                    Detail_Balance_Wi.append(abs(Wi[i,j]-Wi[j,i])/(Wi[i,j]+Wi[j,i]))   
        #             print(abs(A[i,j]-A[j,i])/(A[i,j]+A[j,i])

        ###############  Normalization of Histogram ########################################

        Di_Between.append(np.mean(Detail_Balance_Be)/np.mean(Detail_Balance_Q))
        Di_Within.append(np.mean(Detail_Balance_Wi)/np.mean(Detail_Balance_Q))
#         print(np.mean(Detail_Balance_Be))
    return Di_Between, Di_Within


In [11]:
import networkx as nx
from pygenstability import run, plotting, constructors
import scipy.sparse as sp
import networkx as nx
from pygenstability import run, plotting, constructors
from pygenstability.optimal_scales import identify_optimal_scales
import networkx as nx
import importlib 
import sys
import networkx as nx
from scipy import sparse
# sys.path.insert(0, "/rds/general/user/msaeedia/home/Programs/Brain_neurons")
# import Repository 
# importlib.reload(Repository) 
from pygenstability import run, plotting, constructors
tau=1
freq=30
k_kNN=10
n_Scale=200
n_Tries=600
N_workers=8
Char2=["wild_young","AD_young","old_AD"]
l1=0
l2=0
for l1 in range(3):
    if l1==0:
        count=[0,1,4,5,8,9,10,11,14,15,16]
    if l1==1:
        count=[2,3,6,7,12,13]
    if l1==2:
        count=[0,2,3,4]

    for l2 in range(len(count)):
        spike_trian=np.load('/Users/msaeedia/data/Ann/spike_train/spike_trains_%s_%d_fam1.npy' % (Char2[l1],count[l2]), allow_pickle=True)
        L=len(spike_trian[0])
        N=len(spike_trian)
        #         print(len(spike_trian[0]),len(spike_trian[10]),len(spike_trian[50]),L,N)
        A=np.zeros((N,N))
        print(l1,l2)
        t=0
        for i in range(N):
            Row=np.load('/Users/msaeedia/Results/Ann/Adj_Nod/Adj_Place_Cell_%s_%d_N_%d_tau_fixed_%d_freq_%d_row_%d.npy' % (Char2[l1],count[l2],N,tau,freq,i))
            A[t,:]=Row[:]
            t+=1    
            
        Q=kNN(A,N,k=10)
        adjacency = sparse.csr_matrix(Q)
        # print(adjacency)
        D = nx.to_networkx_graph(Q,create_using=nx.DiGraph())
        
        
        path='/Users/msaeedia/Results/Ann/MS'
        all_results=np.load('%s/all_results_%.1f_Place_Cell_%s_%d_N_%d_fixed_tau_%d_scale_%d_tries_%d_kNN_%d.npy' % (path,freq,Char2[l1],count[l2],N,tau,n_Scale,n_Tries,k_kNN) , allow_pickle=True)
        all_results = all_results.tolist()

        all_results = identify_optimal_scales(all_results,kernel_size=6,window_size=6)

        
        selected_partitions=all_results['selected_partitions']
        S_max=len(selected_partitions)
        Color=[i+1 for i in range(S_max)]
        words=['$S_{%d}$' % (i+1) for i in range(S_max)]
        # print(words)


        Di_Between, Di_Within = PRA_partition(A,Q,D,all_results)
        # Di[0]
        y=Di_Between[0:S_max]
        x=Di_Within[0:S_max]
        plt.scatter(x,y,c=Color)
        # print(Di_Between, Di_Within)


        plt.title(r"%s_%d" % (Char2[l1],count[l2]),fontsize = 20)
        plt.ylabel(r'rPRA($\mathcal{B}$)',fontsize = 20)
        plt.xlabel(r'rPRA($\mathcal{W}$)',fontsize = 20)
        plt.yticks(fontsize=13)
        plt.xticks(fontsize=13)
        # # plt.legend(loc ="upper left", fontsize=15)
        
#         for i, txt in enumerate(words):
#             plt.annotate(txt, (x[i]-.01, y[i]+.02),fontsize = 15)
#         x=[i for i in x if str(i) != 'nan']
#         y=[i for i in y if str(i) != 'nan']
#         print(x,y)
#         plt.ylim((min(y)-.04,  max(y)+.04))
#         plt.xlim((min(x)-.02,  max(x)+.02))
        plt.tight_layout()
        plt.xlim((.1,  1))
        plt.ylim((1,  1.45))
        plt.savefig('/Users/msaeedia/figures/Ann/PRA_communities/Gro_With-Betw_%s_%d_N_%d_tau_%.2f.png' % (Char2[l1],count[l2],N,tau))
        plt.close('all')

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
1 0
1 1
1 2
1 3
1 4
1 5
2 0
2 1
2 2
2 3
